<a href="https://colab.research.google.com/github/Klarret05/SenalesySistemas/blob/main/Klarret_Ejercicio_07_05_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Punto 1***

Reconstruir la señal $x(t) = A * \mathrm{rect}_{\frac{\tau}{2}}(t)$
, por medio de las Series de Fourier.

---




In [ ]:
'''Importar las bibliotecas necesarias'''

# Importar las bibliotecas necesarias
from matplotlib import animation, rc
from IPython.display import HTML
from ipywidgets import interact,IntSlider
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import warnings

# Definir los parámetros de la señal
A = 5  # Amplitud de la señal
T = 10  # Duración de la señal en segundos
Fs = 100  # Frecuencia de muestreo (muestras por segundo)
tv = np.arange(-T/2, T/2, 1/Fs)  # Vector de tiempo
Nm = len(tv)  # Número total de muestras
N = 10  # Número de armónicos a considerar
wo = 2 * np.pi / T  # Frecuencia fundamental

# Generar las funciones base de Fourier
phin = np.zeros((Nm, 2*N+1), dtype=np.complex128)  # Matriz para almacenar las funciones base
for n in range(-N, N+1):
    phin[:, n+N] = np.exp(1j * n * wo * tv)  # Calcular la función base para cada armónico

# Definir la señal de pulso rectangular
tau = T  # Ancho del pulso (igual a la duración de la señal)
xo = A * (np.heaviside(tv + tau/4, 0) - np.heaviside(tv - tau/4, 0))  # Señal de pulso rectangular

# Calcular los coeficientes de Fourier
nv = np.linspace(-N, N, 2*N+1)  # Vector de números armónicos
Cn = ((A) / (nv * np.pi)) * (np.sin(nv * (np.pi/2)))  # Calcular los coeficientes
Cn[N] = A / 2  # Componente DC (continua)

Luego se procede a graficar el espectro:

In [ ]:
#graficar espectro  # Este código grafica el espectro de la señal
fig = plt.figure(figsize=(8,8))  # Crea una figura de 8x8
plt.subplot(2,2,1)  # Crea el primer subgráfico (de 4) en una cuadrícula de 2x2
plt.stem(nv, np.real(Cn), 'r')  # Grafica la parte real de Cn en rojo (eje y) vs nv (eje x)
plt.xlabel(r'$n$',fontsize = 14)  # Etiqueta del eje x: 'n'
plt.ylabel(r'$Re\{C_n\}$',fontsize = 14)  # Etiqueta del eje y: 'Re{Cn}'
plt.axis('tight')  # Ajusta los límites del gráfico
plt.grid()  # Agrega una cuadrícula

plt.subplot(2,2,2)  # Crea el segundo subgráfico
plt.stem(nv, np.imag(Cn), 'r')  # Grafica la parte imaginaria de Cn
plt.xlabel(r'$n$',fontsize = 14)  # Etiqueta del eje x: 'n'
plt.ylabel(r'$Im\{C_n\}$',fontsize = 14)  # Etiqueta del eje y: 'Im{Cn}'
plt.axis('tight')  # Ajusta los límites del gráfico
plt.grid()  # Agrega una cuadrícula

plt.subplot(2,2,3)  # Crea el tercer subgráfico
plt.stem(nv, abs(Cn), 'r')#magnitud del espectro  # Grafica la magnitud de Cn
plt.xlabel(r'$n$',fontsize = 14)  # Etiqueta del eje x: 'n'
plt.ylabel(r'$|C_n|$',fontsize = 14)  # Etiqueta del eje y: '|Cn|'
plt.axis('tight')  # Ajusta los límites del gráfico
plt.grid()  # Agrega una cuadrícula

plt.subplot(2,2,4)  # Crea el cuarto subgráfico
plt.stem(nv, np.angle(Cn), 'r')#fase del espectro  # Grafica la fase de Cn
plt.xlabel(r'$n$',fontsize = 14)  # Etiqueta del eje x: 'n'
plt.ylabel(r'$\langle C_n$',fontsize = 14)  # Etiqueta del eje y: '<Cn'
plt.axis('tight')  # Ajusta los límites del gráfico
plt.grid()  # Agrega una cuadrícula

fig.tight_layout()  # Ajusta el diseño para evitar superposiciones

Por ultimo, elaborar un codigo que permita modular el numero de armonicos:

In [ ]:
Px = A**2 # estimar según señal estudiada
def pltest(Na=1):
    ind = np.arange(N-Na,N+Na+1)
    er = 1- np.sum(abs(Cn[ind])**2)/Px
    #señal reconstruida
    xe = phin[:,ind].dot(Cn[ind] ) # dot calcula el producto entre vectores
    plt.plot(tv,xe,color='b',label='$x_e(t)$') # señal estimada o filtrada
    plt.plot(tv,xo,color='r',label='x(t)') #señal original
    plt.title('$E_r$=%.2f[%%]' % (100*er),fontsize=16)
    plt.xlabel('t[s]')
    plt.ylabel('x(t)')
    plt.grid()
    plt.legend()
    plt.show()

    return
interact(pltest,Na=(1,N,1))

In [ ]:
# Crea la figura y los ejes para la animación
fig, ax = plt.subplots(2, 3,figsize=(12,5))
# Define la función que actualiza cada frame de la animación
def update(n):
    # Limpia todos los subplots antes de dibujar el nuevo frame
    ax[0,0].clear()
    ax[0,1].clear()
    ax[0,2].clear()
    ax[1,0].clear()
    ax[1,1].clear()
    ax[1,2].clear()

    # Calcula los índices de los armónicos a considerar en este frame
    ind = np.arange(N-n,N+n+1)
    # Grafica la magnitud del espectro completo en azul
    ax[0,0].stem(nv, abs(Cn), 'b')
    # Grafica la magnitud de los armónicos seleccionados en rojo
    ax[0,0].stem(nv[ind], abs(Cn[ind]), 'r')
    # Etiqueta el eje x del subplot del espectro
    ax[0,0].set_xlabel(r'$n$[rad/s]')
    # Etiqueta el eje y del subplot del espectro
    ax[0,0].set_ylabel(r'$|C_n|$')

    # Calcula las componentes espectrales para los armónicos seleccionados
    ce = phin[:,ind].dot(np.diag(Cn[ind]) )
    # Grafica la parte real de las componentes espectrales
    ax[0,1].plot(tv,np.real(ce))
    # Etiqueta el eje x del subplot de la parte real
    ax[0,1].set_xlabel('t[s]')
    # Etiqueta el eje y del subplot de la parte real
    ax[0,1].set_ylabel('$Re\{c_n\phi_n(t)\}$')

    # Grafica la parte imaginaria de las componentes espectrales
    ax[0,2].plot(tv,np.imag(ce))
    # Etiqueta el eje x del subplot de la parte imaginaria
    ax[0,2].set_xlabel('t[s]')
    # Etiqueta el eje y del subplot de la parte imaginaria
    ax[0,2].set_ylabel('$Im\{c_n\phi_n(t)\}$')


    # Reconstruye la señal con los armónicos seleccionados
    er = 1- np.sum(abs(Cn[ind])**2)/Px
    # Calcula la señal reconstruida
    xe = phin[:,ind].dot(Cn[ind] )
    # Calcula el error entre la señal original y la reconstruida
    et = xo - xe

    # Grafica la señal original en rojo
    ax[1,1].plot(tv,xo,'r',linewidth=4,label='$x(t)$')
    # Grafica la señal reconstruida en azul
    ax[1,1].plot(tv,xe,color='b',label='$x_e(t)$')
    # Grafica el error en verde
    ax[1,1].plot(tv,et,color='g',label='$e(t)$')
    # Agrega un título al subplot de la reconstrucción mostrando el error
    ax[1,1].set_title('$E_r$=%.2f[%%]' % (100*er))
    # Etiqueta el eje x del subplot de la reconstrucción
    ax[1,1].set_xlabel('t[s]')
    # Etiqueta el eje y del subplot de la reconstrucción
    ax[1,1].set_ylabel('x(t)')
    # Agrega una leyenda al subplot de la reconstrucción
    ax[1,1].legend(loc='center left', bbox_to_anchor=(1, 0.5))
    # Desactiva los ejes de los subplots vacíos
    ax[1,0].axis('off')
    ax[1,2].axis('off')
    # Ajusta el espacio entre los subplots
    plt.subplots_adjust(wspace=0.5,hspace=0.7)
    # Muestra la gráfica
    plt.show()

# Crea la animación llamando a la función update para cada frame
anim = animation.FuncAnimation(fig, update, frames=np.arange(0,N+1,1), blit=False)
# Configura la animación para que se muestre en el notebook
rc('animation', html='jshtml')

# ***Punto 2***

Analizar

---

In [ ]:
from matplotlib import animation, rc
from IPython.display import HTML
from ipywidgets import interact,IntSlider
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import warnings

# Amplitud de la señal
A = 5
# Duración total de la señal en segundos
T = 10
# Frecuencia de muestreo en muestras por segundo
Fs = 100
# Vector de tiempo, desde -T/2 hasta T/2, con pasos de 1/Fs
tv = np.arange(-T/2, T/2, 1/Fs)
# Número total de muestras en el vector de tiempo
Nm = len(tv)
# Número de armónicos a considerar en la serie de Fourier
N = 10
# Frecuencia fundamental (angular) de la señal, calculada como 2π/T
wo = 2 * np.pi / T

# Definición de las funciones base de Fourier
# phin: Matriz para almacenar las funciones base, con dimensiones (Nm, 2N+1)
phin = np.zeros((Nm, 2*N+1), dtype=np.complex128)
# Bucle para generar cada función base (armónico)
for n in range(-N, N+1, 1):
    # Calcula la función base (exponencial compleja) para el armónico 'n'
    # y la almacena en la columna correspondiente de la matriz 'phin'
    phin[:, n+N] = np.exp(1j * n * wo * tv)

# Definición de la señal: Señal triangular centrada en t = 0
# tau: Duración/ancho de la señal, en este caso igual a la duración total (T)
tau = T
# xo: Señal triangular, calculada usando una función por partes
#     - A * (1 - 4 * np.abs(tv) / tau): Define la forma triangular
#     - (np.abs(tv) <= tau / 4):  Asegura que la señal sea cero fuera del intervalo [-tau/4, tau/4]
xo = A * (1 - 4 * np.abs(tv) / tau) * (np.abs(tv) <= tau / 4)

# Cálculo del espectro de la señal (coeficientes de Fourier)
# cn: Vector para almacenar los coeficientes de Fourier, inicializado con ceros
cn = np.zeros(2*N+1, dtype=np.complex128)
# nv: Vector con los números de los armónicos, desde -N hasta N
nv = np.linspace(-N, N, 2*N+1)
# dt: Paso de tiempo, calculado como 1/Fs
dt = 1/Fs
# Bucle para calcular cada coeficiente de Fourier
for i, n in enumerate(range(-N, N+1)):
    # base: Exponencial compleja conjugada para el armónico 'n', usada en el cálculo del coeficiente
    base = np.exp(-1j * n * wo * tv)
    # Cn[i]: Coeficiente de Fourier para el armónico 'n', calculado como la integral
    #        de la señal multiplicada por la base conjugada, normalizada por T
    #        La integral se aproxima usando una suma y se multiplica por dt
    Cn[i] = (1/T) * np.sum(xo * base) * dt

In [ ]:
# Graficar el espectro de la señal
# Crear una figura con un tamaño de 8x8 pulgadas
fig = plt.figure(figsize=(8,8))
# Dividir la figura en una cuadrícula de 2x2 subplots y seleccionar el primer subplot (arriba a la izquierda)
plt.subplot(2,2,1)
# Graficar la parte real de los coeficientes de Fourier (Cn) en función del número de armónico (nv)
# usando una gráfica de tallos ('stem') en color rojo ('r')
plt.stem(nv, np.real(Cn), 'r')
# Etiquetar el eje x como 'n' (número de armónico) con un tamaño de fuente de 14
plt.xlabel(r'$n$',fontsize = 14)
# Etiquetar el eje y como 'Re{Cn}' (parte real de Cn) con un tamaño de fuente de 14
plt.ylabel(r'$Re\{C_n\}$',fontsize = 14)
# Ajustar los límites del gráfico para que se ajusten a los datos
plt.axis('tight')
# Agregar una cuadrícula al gráfico
plt.grid()

# Seleccionar el segundo subplot (arriba a la derecha)
plt.subplot(2,2,2)
# Graficar la parte imaginaria de los coeficientes de Fourier (Cn) en función del número de armónico (nv)
plt.stem(nv, np.imag(Cn), 'r')
# Etiquetar el eje x como 'n'
plt.xlabel(r'$n$',fontsize = 14)
# Etiquetar el eje y como 'Im{Cn}' (parte imaginaria de Cn)
plt.ylabel(r'$Im\{C_n\}$',fontsize = 14)
# Ajustar los límites del gráfico
plt.axis('tight')
# Agregar una cuadrícula
plt.grid()

# Seleccionar el tercer subplot (abajo a la izquierda)
plt.subplot(2,2,3)
# Graficar la magnitud de los coeficientes de Fourier (Cn) en función del número de armónico (nv)
plt.stem(nv, abs(Cn), 'r')#magnitud del espectro
# Etiquetar el eje x como 'n'
plt.xlabel(r'$n$',fontsize = 14)
# Etiquetar el eje y como '|Cn|' (magnitud de Cn)
plt.ylabel(r'$|C_n|$',fontsize = 14)
# Ajustar los límites del gráfico
plt.axis('tight')
# Agregar una cuadrícula
plt.grid()

# Seleccionar el cuarto subplot (abajo a la derecha)
plt.subplot(2,2,4)
# Graficar la fase de los coeficientes de Fourier (Cn) en función del número de armónico (nv)
plt.stem(nv, np.angle(Cn), 'r')#fase del espectro
# Etiquetar el eje x como 'n'
plt.xlabel(r'$n$',fontsize = 14)
# Etiquetar el eje y como '<Cn' (fase de Cn)
plt.ylabel(r'$\langle C_n$',fontsize = 14)
# Ajustar los límites del gráfico
plt.axis('tight')
# Agregar una cuadrícula
plt.grid()

# Ajustar el diseño de la figura para evitar que los subplots se superpongan
fig.tight_layout()

In [ ]:
# Px: Potencia de la señal original, estimada como A^2 (ajustar según la señal)
Px = A**2
# Definir la función 'pltest' que permite visualizar la reconstrucción de la señal
# con un número variable de armónicos (Na)
def pltest(Na=1):
    # ind: Índices de los armónicos a considerar en la reconstrucción,
    #      desde N-Na hasta N+Na, creando un rango simétrico alrededor del armónico central (N)
    ind = np.arange(N-Na,N+Na+1)
    # er: Error relativo de la reconstrucción, calculado como 1 menos la
    #     suma de las potencias de los armónicos seleccionados dividida por la potencia total (Px)
    er = 1- np.sum(abs(Cn[ind])**2)/Px
    # Reconstrucción de la señal (xe) usando los armónicos seleccionados
    # phin[:,ind]: Selecciona las columnas de 'phin' correspondientes a los armónicos seleccionados
    # Cn[ind]: Selecciona los coeficientes de Fourier correspondientes a los armónicos seleccionados
    # .dot(): Realiza el producto punto entre las funciones base y los coeficientes para obtener la señal reconstruida
    xe = phin[:,ind].dot(Cn[ind] )
    # Graficar la señal reconstruida (xe) en azul ('b') con la etiqueta '$x_e(t)$'
    plt.plot(tv,xe,color='b',label='$x_e(t)$')
    # Graficar la señal original (xo) en rojo ('r') con la etiqueta 'x(t)'
    plt.plot(tv,xo,color='r',label='x(t)')
    # Agregar un título al gráfico que muestra el error relativo (er) en porcentaje
    plt.title('$E_r$=%.2f[%%]' % (100*er),fontsize=16)
    # Etiquetar el eje x como 't[s]' (tiempo en segundos)
    plt.xlabel('t[s]')
    # Etiquetar el eje y como 'x(t)' (señal en función del tiempo)
    plt.ylabel('x(t)')
    # Agregar una cuadrícula al gráfico
    plt.grid()
    # Mostrar una leyenda para identificar las señales en el gráfico
    plt.legend()
    # Mostrar el gráfico
    plt.show()

    return # La función regresa None (opcional)

# Crear un widget interactivo para modificar el número de armónicos (Na)
# y visualizar la reconstrucción de la señal en tiempo real
# interact(): Crea el widget interactivo
# pltest: Función que se llama cada vez que se modifica el valor del widget
# Na=(1, N, 1): Define el rango del widget (desde 1 hasta N, con pasos de 1)
interact(pltest,Na=(1,N,1))

In [ ]:
# Crear la figura y los ejes para la animación
fig, ax = plt.subplots(2, 3,figsize=(12,5))
# Definir la función que actualiza cada frame de la animación
def update(n):
    # Limpiar todos los subplots antes de dibujar el nuevo frame
    ax[0,0].clear()
    ax[0,1].clear()
    ax[0,2].clear()
    ax[1,0].clear()
    ax[1,1].clear()
    ax[1,2].clear()

    # Calcular los índices de los armónicos a considerar en este frame
    ind = np.arange(N-n,N+n+1)
    # Graficar la magnitud del espectro completo en azul
    ax[0,0].stem(nv, abs(Cn), 'b')
    # Graficar la magnitud de los armónicos seleccionados en rojo
    ax[0,0].stem(nv[ind], abs(Cn[ind]), 'r')
    # Etiquetar el eje x del subplot del espectro
    ax[0,0].set_xlabel(r'$n$[rad/s]')
    # Etiquetar el eje y del subplot del espectro
    ax[0,0].set_ylabel(r'$|C_n|$')

    # Calcular las componentes espectrales para los armónicos seleccionados
    ce = phin[:,ind].dot(np.diag(Cn[ind]) )
    # Graficar la parte real de las componentes espectrales
    ax[0,1].plot(tv,np.real(ce))
    # Etiquetar el eje x del subplot de la parte real
    ax[0,1].set_xlabel('t[s]')
    # Etiquetar el eje y del subplot de la parte real
    ax[0,1].set_ylabel('$Re\{c_n\phi_n(t)\}$')

    # Graficar la parte imaginaria de las componentes espectrales
    ax[0,2].plot(tv,np.imag(ce))
    # Etiquetar el eje x del subplot de la parte imaginaria
    ax[0,2].set_xlabel('t[s]')
    # Etiquetar el eje y del subplot de la parte imaginaria
    ax[0,2].set_ylabel('$Im\{c_n\phi_n(t)\}$')


    # Reconstruir la señal con los armónicos seleccionados
    er = 1- np.sum(abs(Cn[ind])**2)/Px
    # Calcular la señal reconstruida
    xe = phin[:,ind].dot(Cn[ind] )
    # Calcular el error entre la señal original y la reconstruida
    et = xo - xe

    # Graficar la señal original en rojo
    ax[1,1].plot(tv,xo,'r',linewidth=4,label='$x(t)$')
    # Graficar la señal reconstruida en azul
    ax[1,1].plot(tv,xe,color='b',label='$x_e(t)$')
    # Graficar el error en verde
    ax[1,1].plot(tv,et,color='g',label='$e(t)$')
    # Agregar un título al subplot de la reconstrucción mostrando el error
    ax[1,1].set_title('$E_r$=%.2f[%%]' % (100*er))
    # Etiquetar el eje x del subplot de la reconstrucción
    ax[1,1].set_xlabel('t[s]')
    # Etiquetar el eje y del subplot de la reconstrucción
    ax[1,1].set_ylabel('x(t)')
    # Agregar una leyenda al subplot de la reconstrucción
    ax[1,1].legend(loc='center left', bbox_to_anchor=(1, 0.5))
    # Desactivar los ejes de los subplots vacíos
    ax[1,0].axis('off')
    ax[1,2].axis('off')
    # Ajustar el espacio entre los subplots
    plt.subplots_adjust(wspace=0.5,hspace=0.7)
    # Mostrar la gráfica
    plt.show()

# Crear la animación llamando a la función update para cada frame
anim = animation.FuncAnimation(fig, update, frames=np.arange(0,N+1,1), blit=False)
# Configurar la animación para que se muestre en el notebook
rc('animation', html='jshtml')